In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path 
from gemd import MaterialTemplate, ProcessTemplate, MeasurementTemplate, ParameterTemplate, RealBounds
from openmsimodel.workflow.workflow import Workflow
from openmsimodel.subworkflow.process_block import ProcessBlock
from openmsimodel.entity.gemd.material import Material
from openmsimodel.entity.gemd.process import Process
from openmsimodel.entity.gemd.measurement import Measurement
from openmsimodel.entity.gemd.ingredient import Ingredient
from openmsimodel.db.open_db import OpenDB
from openmsimodel.graph.open_graph import OpenGraph

In [2]:
workflow = Workflow()
# w = AssemblyChain
# AssemblyNetwork
# discovery_platform
# laboratory

## path
# sequence
# synthesis

alloy_ingredient = Ingredient("Alloy Ingredient")
polishing_process = Process("Polishing", template=ProcessTemplate("Heating"))
polished_alloy = Material("Polished Alloy", template=MaterialTemplate("Alloy"))
polishing_block = ProcessBlock(
    name=f"Polishing Alloy",
    workflow=workflow,
    material=polished_alloy,
    ingredients=[alloy_ingredient],
    process=polishing_process,
    measurements=[],
)
polishing_block.link_within()

polished_alloy_ingredient = Ingredient("Polished Alloy Ingredient")
heating_process = Process("Heating", template=ProcessTemplate("Heating", parameters=ParameterTemplate(
    name="Temperature",
    bounds=RealBounds(0, 1500, "Kelvin"),
)))
heated_alloy = Material("Heated Alloy", template=MaterialTemplate("Alloy"))
heating_block = ProcessBlock(
    name=f"Heating Alloy",
    workflow=workflow,
    material=heated_alloy,
    ingredients=[polished_alloy_ingredient],
    process=heating_process,
    measurements=[],
)
heating_block.link_within() 
heating_block.link_prior(polishing_block, ingredient_name_to_link="Polished Alloy Ingredient")


In [3]:
to_be_visualized = heating_block.gemd_assets
output = str(Path().absolute() / "output")
print(output)
open_graph = OpenGraph(name="heating", source=to_be_visualized, output=output, which='all')
G, relabeled_G, name_mapping = open_graph.build_graph()

/srv/hemi01-j01/openmsimodel/examples/materials_data/output
-- Building alls of list with 8 items
Extracting list...
Done.
Relabeling nodes...
Saving graphs...
Couldn't find GraphViz graph. Can't dump SVG and DOT files.
Dumping graphml...
Time elapsed: 0.008364200592041016
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/materials_data/output/heating_all.dot, /srv/hemi01-j01/openmsimodel/examples/materials_data/output/heating_all.svg and /srv/hemi01-j01/openmsimodel/examples/materials_data/output/heating_all.graphml
-- Analysis --
cycles in the graph: []
disregarded/total number of gemd objects: 0/8
total nb of isolates in the graph: 0


# ProcessBlock

In [4]:
print(polishing_process.assets)

[{'uids': {'auto': '2f054d6d-808f-409a-839c-d282cb0dd9c3'}, 'conditions': [], 'parameters': [], 'tags': [], 'description': None, 'allowed_labels': [], 'allowed_names': [], 'name': 'Heating', 'type': 'process_template'}, {'uids': {'auto': '321d5b3d-156d-4aa1-b19a-e5f7ea3f86b0'}, 'conditions': [], 'parameters': [], 'tags': [], 'notes': None, 'file_links': [], 'template': {'uids': {'auto': '2f054d6d-808f-409a-839c-d282cb0dd9c3'}, 'conditions': [], 'parameters': [], 'tags': [], 'description': None, 'allowed_labels': [], 'allowed_names': [], 'name': 'Heating', 'type': 'process_template'}, 'name': 'Polishing', 'type': 'process_spec', 'ingredients': ["<IngredientSpec 'Alloy Ingredient'>"], 'output_material': "<MaterialSpec 'Polished Alloy'>"}, {'uids': {'auto': '28068e34-2ab2-4a80-bf82-75100565c8d2'}, 'conditions': [], 'parameters': [], 'tags': [], 'notes': None, 'file_links': [], 'spec': {'uids': {'auto': '321d5b3d-156d-4aa1-b19a-e5f7ea3f86b0'}, 'conditions': [], 'parameters': [], 'tags': []

In [5]:
block = ProcessBlock.from_spec_or_run(
    str(polishing_process.name + "_backward"), notes=None, spec=polishing_process.spec, run=polishing_process.run
)

Test
{'process': {'uids': {'auto': '28068e34-2ab2-4a80-bf82-75100565c8d2'}, 'conditions': [], 'parameters': [], 'tags': [], 'notes': None, 'file_links': [], 'spec': {'uids': {'auto': '321d5b3d-156d-4aa1-b19a-e5f7ea3f86b0'}, 'conditions': [], 'parameters': [], 'tags': [], 'notes': None, 'file_links': [], 'template': {'uids': {'auto': '2f054d6d-808f-409a-839c-d282cb0dd9c3'}, 'conditions': [], 'parameters': [], 'tags': [], 'description': None, 'allowed_labels': [], 'allowed_names': [], 'name': 'Heating', 'type': 'process_template'}, 'name': 'Polishing_backward', 'type': 'process_spec', 'ingredients': ["<IngredientSpec 'Alloy Ingredient'>"], 'output_material': "<MaterialSpec 'Polished Alloy'>"}, 'name': 'Polishing_backward', 'source': None, 'type': 'process_run', 'ingredients': ["<IngredientRun 'Alloy Ingredient'>"], 'output_material': "<MaterialRun 'Polished Alloy'>"}, 'uids': {'auto': '21c0e6f6-0863-41b0-ad5f-36aebd3df789'}, 'number_fraction': None, 'tags': [], 'volume_fraction': None, '

NameError: ingredients must have unique names. Found a duplicate: Alloy Ingredient